# Name of Project Here

Team members: Lauren Chin and Lauren Nguyen

possible datasets: 
* [Alzheimer's Dataset](https://www.kaggle.com/datasets/rabieelkharoua/alzheimers-disease-dataset)
* [Breast Cancer Dataset](https://www.kaggle.com/datasets/saurabhbadole/breast-cancer-wisconsin-state)
* [General Cancer Dataset](https://www.kaggle.com/datasets/rabieelkharoua/cancer-prediction-dataset)
* [Asthma Dataset](https://www.kaggle.com/datasets/rabieelkharoua/asthma-disease-dataset)



## Dataset

* Source:
* Format:
* Contents:
* Attributes:


## Implementation/Technical Merit

### Anticipated Challenges
* Pre-processing:
* Classification: 


## Potential Impact of Results

* Why these results are useful:
    * We are looking at the asthma dataset with the goal of creating classifiers to predict if someone has asthma or not
    * These results are useful because it can help with medical diagnosis
    * While asthma isn't as life threatening as a disease like cancer, detection is still important so measures can be taken to help ease symptoms for patients

* Stakeholders:
    * Doctors (people who have to make the medical diagnosis)
    * Patients (people who could potentially have asthma)
